In [ ]:
import os
from timeit import default_timer as timer
import pandas as pd
from datetime import datetime, timedelta
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd

In [ ]:
path_to_data='/scratch/spf248/covid/data'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
day=list(range(8,20))
night=list(range(8))+list(range(20,24))
inputs_folders=sorted([x.split('/')[-1][:-2] for x in glob(os.path.join(path_to_data,source,'s3',country,'*'))])
start_date=pd.to_datetime(inputs_folders[0],format='%Y%m%d').strftime('%m/%d/%Y')
end_date=pd.to_datetime(inputs_folders[-1],format='%Y%m%d').strftime('%m/%d/%Y')
print('Start:',start_date)
print('End:',end_date)

# Compute share of activity in unique neighborhoods

In [ ]:
def get_coords(df):
    df['latitude']=df['point'].apply(lambda x:x['latitude'])
    df['longitude']=df['point'].apply(lambda x:x['longitude'])
    return df.drop('point',1)

In [ ]:
print('Import')
start = timer()
data_dirs=glob(os.path.join(path_to_data,source,'processed',country,'pings_geocoded','*'))
pings_geocoded = pd.concat(
    pd.read_parquet(parquet_file)
    for data_dir in data_dirs for parquet_file in Path(data_dir).glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Process')
start = timer()
pings_geocoded['date']=pd.to_datetime(pings_geocoded.time.dt.date)
pings_geocoded['hour']=pings_geocoded.time.dt.hour
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Count neighborhood visits')
start = timer()
n_admin_date_id=pings_geocoded.groupby(['date','cuebiq_id'])[admin_id].agg(['nunique','count'])
n_admin_daytime_id=pings_geocoded[pings_geocoded.hour.isin(day)].groupby(['date','cuebiq_id'])[admin_id].agg(['nunique','count'])
n_admin_nighttime_id=pings_geocoded[pings_geocoded.hour.isin(night)].groupby(['date','cuebiq_id'])[admin_id].agg(['nunique','count'])
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Process')
start = timer()
n_admin_date_id['pct_unique']=n_admin_date_id['nunique'].divide(n_admin_date_id['count'])
n_admin_daytime_id['pct_unique']=n_admin_daytime_id['nunique'].divide(n_admin_daytime_id['count'])
n_admin_nighttime_id['pct_unique']=n_admin_nighttime_id['nunique'].divide(n_admin_nighttime_id['count'])
n_admin_date_id=n_admin_date_id[n_admin_date_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
n_admin_daytime_id=n_admin_daytime_id[n_admin_daytime_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
n_admin_nighttime_id=n_admin_nighttime_id[n_admin_nighttime_id.index.get_level_values('date').isin(pd.date_range(start_date,end_date))].copy()
n_admin_date_id.rename(columns={'count':'n_pings'},inplace=True)
n_admin_daytime_id.rename(columns={'count':'n_pings'},inplace=True)
n_admin_nighttime_id.rename(columns={'count':'n_pings'},inplace=True)
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Save')
start = timer()
n_admin_date_id.to_csv(os.path.join(path_to_data,source,'processed',country,'n_admin_date_id.csv'))
n_admin_daytime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'n_admin_daytime_id.csv'))
n_admin_nighttime_id.to_csv(os.path.join(path_to_data,source,'processed',country,'n_admin_nighttime_id.csv'))
print("Done in", round(timer()-start), "sec")

# Figures